## CmdStanPyの動作確認

### CmdStanPy公式の"Hello World"

In [1]:
# import packages
import os
from cmdstanpy import cmdstan_path, CmdStanModel

# specify Stan program file
bernoulli_stan = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.stan')

# instantiate the model; compiles the Stan program as needed.
bernoulli_model = CmdStanModel(stan_file=bernoulli_stan)

# inspect model object
print(bernoulli_model)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /root/.cmdstan/cmdstan-2.26.1/examples/bernoulli/bernoulli


CmdStanModel: name=bernoulli
	 stan_file=/root/.cmdstan/cmdstan-2.26.1/examples/bernoulli/bernoulli.stan
	 exe_file=/root/.cmdstan/cmdstan-2.26.1/examples/bernoulli/bernoulli
	 compiler_options=stanc_options=None, cpp_options=None


In [2]:
# specify data file
bernoulli_data = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')

# fit the model
bern_fit = bernoulli_model.sample(data=bernoulli_data, output_dir='.')

# printing the object reports sampler commands, output files
print(bern_fit)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 4


CmdStanMCMC: model=bernoulli chains=4['method=sample', 'algorithm=hmc', 'adapt', 'engaged=1']
 csv_files:
	/workdir/demo/bernoulli-202103070614-1.csv
	/workdir/demo/bernoulli-202103070614-2.csv
	/workdir/demo/bernoulli-202103070614-3.csv
	/workdir/demo/bernoulli-202103070614-4.csv
 output_files:
	/workdir/demo/bernoulli-202103070614-1-stdout.txt
	/workdir/demo/bernoulli-202103070614-2-stdout.txt
	/workdir/demo/bernoulli-202103070614-3-stdout.txt
	/workdir/demo/bernoulli-202103070614-4-stdout.txt


In [3]:
bern_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-7.30,0.0200,0.76,-8.900,-7.00,-6.80,1400.0,590.0,1.0
theta,0.25,0.0033,0.12,0.075,0.23,0.46,1300.0,560.0,1.0


In [4]:
bern_fit.diagnose()

INFO:cmdstanpy:Processing csv files: /workdir/demo/bernoulli-202103070614-1.csv, /workdir/demo/bernoulli-202103070614-2.csv, /workdir/demo/bernoulli-202103070614-3.csv, /workdir/demo/bernoulli-202103070614-4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory for all transitions.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.


'Processing csv files: /workdir/demo/bernoulli-202103070614-1.csv, /workdir/demo/bernoulli-202103070614-2.csv, /workdir/demo/bernoulli-202103070614-3.csv, /workdir/demo/bernoulli-202103070614-4.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\nNo divergent transitions found.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory for all transitions.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete, no problems detected.'

### 8schools
- pystanとの速度比較

In [5]:
%%time
model = CmdStanModel(stan_file="8schools.stan")

INFO:cmdstanpy:compiling stan program, exe file: /workdir/demo/8schools
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /workdir/demo/8schools


CPU times: user 5.14 ms, sys: 4.22 ms, total: 9.37 ms
Wall time: 17.5 s


- 17~19s程度でコンパイルできた。
- pystan2では1min超を要したから、確かにコンパイルはかなり高速と言える。
  - コンパイルしたモデルは保存されており、2回目以降は数秒で読み込める。(`8schools`と`8schools.hpp`を削除すれば初回同様となる)

In [6]:
%%time
data = "8schools.data.json"
fit = model.sample(data=data, iter_sampling=1000, iter_warmup=500)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


CPU times: user 48.1 ms, sys: 30.6 ms, total: 78.7 ms
Wall time: 238 ms


- 200ms強を要しており、微差ではあるがPyStan2（163ms）よりやや遅い？

In [7]:
fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-4.900,0.083,2.70,-9.80,-4.700,-0.82,1100.0,5100.0,1.0
mu,8.100,0.170,5.50,-0.73,8.000,17.00,1100.0,5200.0,1.0
tau,6.900,0.180,5.80,0.50,5.500,18.00,1000.0,4900.0,1.0
eta[1],0.400,0.014,0.94,-1.20,0.420,2.00,4500.0,21740.0,1.0
eta[2],-0.013,0.014,0.86,-1.40,-0.012,1.40,3822.0,18462.0,1.0
eta[3],-0.210,0.014,0.93,-1.70,-0.210,1.30,4632.0,22375.0,1.0
eta[4],-0.051,0.013,0.90,-1.50,-0.033,1.40,4611.0,22274.0,1.0
eta[5],-0.360,0.014,0.87,-1.80,-0.380,1.10,3701.0,17877.0,1.0
eta[6],-0.220,0.013,0.91,-1.70,-0.250,1.30,4584.0,22147.0,1.0


In [8]:
fit.diagnose()

INFO:cmdstanpy:Processing csv files: /tmp/tmpsbryf0sb/8schools-202103070615-1-fwk5f3py.csv, /tmp/tmpsbryf0sb/8schools-202103070615-2-nwo4slhc.csv, /tmp/tmpsbryf0sb/8schools-202103070615-3-0it5i9jm.csv, /tmp/tmpsbryf0sb/8schools-202103070615-4-bbt_piu0.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory for all transitions.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.


'Processing csv files: /tmp/tmpsbryf0sb/8schools-202103070615-1-fwk5f3py.csv, /tmp/tmpsbryf0sb/8schools-202103070615-2-nwo4slhc.csv, /tmp/tmpsbryf0sb/8schools-202103070615-3-0it5i9jm.csv, /tmp/tmpsbryf0sb/8schools-202103070615-4-bbt_piu0.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\nNo divergent transitions found.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory for all transitions.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete, no problems detected.'